In [3]:
from DataProcess.utils.filehelper import read_csv_file
from tqdm import tqdm
import csv

goods_same = read_csv_file('./data/goods_same.csv')[1:]
goods_similar = read_csv_file('./data/goods_similar_correct.csv')[1:]

1233383it [00:03, 396392.31it/s]
104171it [00:00, 328142.61it/s]


In [9]:
from DataProcess.utils.filehelper import join_prompt_strings
# 有422个长度超过256

final_data = []
max_length = 0
num_max_length = 0
now_str = ''
id = 0

for data in tqdm(goods_similar):
    if len(data)==7:
        if data[5] == 1:
            pass
        else:
            now_str = join_prompt_strings(prompt="对比相同和相似物资:",str_a = data[3],str_b = data[4])
            if len(now_str)>256:
                num_max_length+=1
            if len(now_str)>max_length:
                max_length = len(now_str)
            else:
                final_data.append([id, now_str,data[5],data[6]])
                id += 1

for data in tqdm(goods_same):
    if len(data) == 4:
        now_str = join_prompt_strings(prompt="对比相同和相似物资:",str_a = data[0],str_b = data[1])
        if len(now_str)>256:
                num_max_length+=1
        if len(now_str)>max_length:
            max_length = len(now_str)
        else:
            final_data.append([id,now_str,data[2],data[3]])
            id += 1
print(num_max_length)

100%|██████████| 1233382/1233382 [00:04<00:00, 265237.95it/s]

422


In [10]:
from sklearn.model_selection import train_test_split

def split_data(data_list, test_size=0.1,stratify=False, random_state=42):
    X = [data[0:2] for data in data_list]
    y = [data[2:] for data in data_list]
    if stratify:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    train_list = [x + list(y) for x, y in zip(X_train, y_train)]
    test_list = [x + list(y) for x, y in zip(X_test, y_test)]
    return train_list, test_list

def export_to_csv(data_list, csv_path):
    with open(csv_path, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['id','data', 'same', 'similar'])  # 表头
        for i, data in enumerate(data_list):
            writer.writerow(data)

train_data, test_data  =  split_data(final_data, test_size=0.1,stratify=True, random_state=114514)

In [11]:
export_to_csv(train_data,'./processed/train.csv')
export_to_csv(test_data, './processed/test.csv')

# Predict

In [1]:
from pybert.model.bert_for_multi_label import BertForMultiLable
import torch
from transformers import BertTokenizer
from services.predict import data_predict

# 获取当前可用的CUDA设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForMultiLable.from_pretrained('T:\WorkSpace\\NLP\Bert-Multi-Label-Text-Classification\pybert\output\checkpoints\\bert', num_labels=2).to(device)

tokenizer = BertTokenizer('T:\WorkSpace\\NLP\Bert-Multi-Label-Text-Classification\pybert\pretrain\\bert\\bert-uncased\\vocab.txt', True)

E:\Anaconda\envs\Bert-Classification\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
examples, predict_result = data_predict(str_a='FORANT笔式pH测试仪871174370~14pH1台',
        str_b='BANTE/般特六合一水质检测仪M852-1~15pH1台',
        model=model,tokenizer=tokenizer,
        max_seq_len=512,example_type='predict',device=device)
print(predict_result)

tensor([[0.0160, 0.5531]], device='cuda:0')


In [3]:
from tqdm import tqdm

num_times = 1000

s = 's'
for i in tqdm(range(num_times)):
    # 发送POST请求并获取响应
   examples, predict_result = data_predict(str_a='FORANT笔式pH测试仪871174370~14pH1台'+i*s,
        str_b='BANTE/般特六合一水质检测仪M852-1~15pH1台',
        model=model,tokenizer=tokenizer,
        max_seq_len=512,example_type='predict',device=device)

100%|██████████| 1000/1000 [00:14<00:00, 67.82it/s]


In [9]:
import requests
from tqdm import tqdm

num_times = 1000
# 修改为您的API地址，如果在本地运行，可以使用 http://127.0.0.1:8000/predict
url = "http://localhost:8000/predict"

# 定义要发送的数据
data = {
    "goods_a": "商品描述A",
    "goods_b": "商品描述B"
}

for i in tqdm(range(num_times)):
    # 发送POST请求并获取响应
    response = requests.post(url, json=data)

  0%|          | 5/1000 [00:14<48:28,  2.92s/it]  


KeyboardInterrupt: 